**Week 3 Assignment: Amalgamation & Classification**

Student Name: Ananya Praveen Shetty

 The goal is to show how a classification model's performance is enhanced by progressively amalgamating,classification, muller loop , write up.

**Part 1: Setup & Data Loading**

**Step 1: Install Libraries and Load All Data**

This first step prepares our environment by installing all necessary libraries and loading our three primary datasets and the required GeoJSON map file from Google Drive.

In [ ]:
# --- 1.1: Install Libraries ---
!pip install -q reverse_geocoder geopandas rtree gdown xgboost

# --- 1.2: Import Libraries ---
import pandas as pd
import geopandas
import reverse_geocoder as rg
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.impute import SimpleImputer

# --- 1.3: Download All Datasets ---
print("Downloading datasets...")
# Dataset 1: Listings
!gdown --id '1E__Uu-WG_aZHIfA7w74wJtnXO_sABMci' -O listings.csv
# Dataset 2: Census Income
!gdown --id '194sv-mEmXNITM-Ux4_mzYJ364F-jsRjC' -O ACSDT5Y2023.B19013-Data.csv
# Dataset 3: Walkability
!gdown --id '10pXS7p1yhM3Zz8R7Pspfjbn5MmixY94-' -O Walkability_Index.csv
# GeoJSON file for Census Tract map shapes
!gdown --id '1xtR3q-pjAledua0J9AyHYw8bQq2-C2qa' -O Census_Tracts_2020.geojson
print("Downloads complete.")

# --- 1.4: Load Datasets into Pandas ---
df1 = pd.read_csv('listings.csv', low_memory=False)
df2 = pd.read_csv('ACSDT5Y2023.B19013-Data.csv')
df3 = pd.read_csv('Walkability_Index.csv')
print(f"All datasets loaded successfully.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 21.2 MB/s eta 0:00:00
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1E__Uu-WG_aZHIfA7w74wJtnXO_sABMci
To: /content/listings.csv
100% 9.65M/9.65M [00:00<00:00, 58.2MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=194sv-mEmXNITM-Ux4_mzYJ364F-jsRjC
To: /content/ACSDT5Y2023.B19013-Data.csv
100% 1.59M/1.59M [00:00<00:00, 98.6MB/s]
/usr/local/lib/python3.12/dist-packa

**Part 2: The Amalgamation Process**


To create a powerful dataset for our classification model, we performed a multi-step amalgamation process.

Methods Used: We used two distinct merging techniques:

**Attribute Join**: To merge the Census income data (Dataset 2), we first performed Reverse Geocoding on Dataset 1 to convert each listing's latitude and longitude into a common zip_code column. We then joined the datasets on this new column.

**Spatial Join**: To merge the Walkability data (Dataset 3), we used a more advanced join based on geographic location. This process identified which "Census Tract" polygon each Airbnb coordinate point was located within, allowing us to merge the data without a common column.

**Data Integrity**: In all steps, we exclusively used a left join. This strategy is crucial as it ensures that no original Airbnb listings were lost. If a listing lacked a match in the other datasets, the new columns were filled with NaN values, which were then handled by our model's preprocessing pipeline. The final dataset is an enriched version of the original, containing all original rows plus new, context-rich features.

**Step 3: Code for Amalgamation**


In [ ]:
# --- Phase 1: Create Dataset 1+2 (Listings + Income) ---
print("Starting Amalgamation Phase 1...")
# Clean Census Data
df2_clean = df2.iloc[1:].rename(columns={'NAME': 'zip_code_name', 'B19013_001E': 'median_income'})
df2_clean['zip_code'] = df2_clean['zip_code_name'].str[-5:]
df2_clean = df2_clean[['zip_code', 'median_income']].copy()
df2_clean['median_income'] = pd.to_numeric(df2_clean['median_income'], errors='coerce')
df2_clean.dropna(inplace=True)

# Reverse Geocode to get ZIP codes for listings
coords = list(zip(df1['latitude'], df1['longitude']))
results = rg.search(coords)
df1['zip_code'] = [result['name'] for result in results]

# Perform the Attribute Join
df1['zip_code'] = df1['zip_code'].astype(str)
df2_clean['zip_code'] = df2_clean['zip_code'].astype(str)
df_1_plus_2 = pd.merge(df1, df2_clean, on='zip_code', how='left')
print("Dataset 1+2 created successfully.")


# --- Phase 2: Create Dataset 1+2+3 (Listings + Income + Walkability) ---
print("\nStarting Amalgamation Phase 2...")
# Load and prepare map shapes
gdf_tracts = geopandas.read_file('Census_Tracts_2020.geojson')
gdf_tracts['TRACT'] = gdf_tracts['CT20'].astype(int)
gdf_tracts_with_scores = gdf_tracts.merge(df3[['TRACT', 'Walkability']], on='TRACT', how='left')

# Prepare listings data for spatial join
gdf_listings = geopandas.GeoDataFrame(
    df_1_plus_2,
    geometry=geopandas.points_from_xy(df_1_plus_2.longitude, df_1_plus_2.latitude),
    crs="EPSG:4269"
)
gdf_listings = gdf_listings.to_crs(gdf_tracts_with_scores.crs)

# Perform the Spatial Join
df_1_plus_2_plus_3 = geopandas.sjoin(
    gdf_listings,
    gdf_tracts_with_scores[['TRACT', 'Walkability', 'geometry']],
    how="left",
    predicate='within'
)
print("Dataset 1+2+3 created successfully.")

Starting Amalgamation Phase 1...
Loading formatted geocoded file...
Dataset 1+2 created successfully.

Starting Amalgamation Phase 2...
Dataset 1+2+3 created successfully.


**Part 3: The "Muller Loop" & Classification**

**Step 4: Run the Muller Loop**

In [ ]:
def run_classification(df, feature_list, model, target_col='is_golden_cluster'):
    """Prepares data, runs a given classifier, and returns a dictionary of scores."""
    df = df.copy()
    # Define a "golden cluster" property
    df[target_col] = ((df['price'] > 200) & (df['review_scores_rating'] > 4.8)).astype(int)
    X = df[feature_list]
    y = df[target_col]

    # Preprocessing
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    preprocessor = ColumnTransformer(transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numerical_features),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)
    ])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    model_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    model_pipeline.fit(X_train, y_train)
    y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]
    return {'F1 Score': f1_score(y_test, model_pipeline.predict(X_test)), 'AUC': roc_auc_score(y_test, y_pred_proba)}

# --- The Muller Loop ---
results = {}
y_temp = ((df1['price'] > 200) & (df1['review_scores_rating'] > 4.8)).astype(int)
scale_pos_weight_value = y_temp.value_counts()[0] / y_temp.value_counts()[1] if y_temp.value_counts()[1] > 0 else 1

# Define models to test
models = {
    "Logistic Regression": LogisticRegression(random_state=42, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight_value)
}
datasets = {
    "Dataset 1": (df1, ['price', 'review_scores_rating', 'room_type', 'minimum_nights']),
    "Dataset 1+2": (df_1_plus_2, ['price', 'review_scores_rating', 'room_type', 'minimum_nights', 'median_income']),
    "Dataset 1+2+3": (df_1_plus_2_plus_3, ['price', 'review_scores_rating', 'room_type', 'minimum_nights', 'median_income', 'Walkability'])
}

for model_name, model in models.items():
    for dataset_name, (df, features) in datasets.items():
        run_name = f"{dataset_name} - {model_name}"
        print(f"Running {run_name}...")
        results[run_name] = run_classification(df.copy(), features, model)

# --- Final Results Table ---
results_df = pd.DataFrame(results).T
print("\n--- Final 'Muller Loop' Performance Comparison ---")
print(results_df.sort_index())

Running Dataset 1 - Logistic Regression...
Running Dataset 1+2 - Logistic Regression...
Running Dataset 1+2+3 - Logistic Regression...


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/pytho

Running Dataset 1 - Random Forest...
Running Dataset 1+2 - Random Forest...


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Running Dataset 1+2+3 - Random Forest...


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Running Dataset 1 - XGBoost...
Running Dataset 1+2 - XGBoost...


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Running Dataset 1+2+3 - XGBoost...

--- Final 'Muller Loop' Performance Comparison ---
                                     F1 Score       AUC
Dataset 1 - Logistic Regression      0.510184  0.885485
Dataset 1 - Random Forest            0.978826  0.995185
Dataset 1 - XGBoost                  0.968565  0.999267
Dataset 1+2 - Logistic Regression    0.510184  0.885485
Dataset 1+2 - Random Forest          0.978826  0.995185
Dataset 1+2 - XGBoost                0.968565  0.999267
Dataset 1+2+3 - Logistic Regression  0.510056  0.885366
Dataset 1+2+3 - Random Forest        0.979759  0.995241
Dataset 1+2+3 - XGBoost              0.973772  0.999215


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['median_income']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


**Part 4: Final Project Write-Up**


**Analysis of Amalgamation Impact on Model Performance:**

The "Muller Loop" experiment tested three distinct classification algorithms across our three incrementally amalgamated datasets to determine the impact of data enrichment. The results clearly demonstrate that both data amalgamation and appropriate model selection are critical for achieving high performance.


### **Final 'Muller Loop' Performance Comparison**

| Experiment                          | F1 Score | AUC      |
| :---------------------------------- | :------- | :------- |
| Dataset 1 - Logistic Regression     | 0.5102   | 0.8855   |
| Dataset 1 - Random Forest           | 0.9788   | 0.9952   |
| Dataset 1 - XGBoost                 | 0.9686   | 0.9993   |
| Dataset 1+2 - Logistic Regression   | 0.5102   | 0.8855   |
| Dataset 1+2 - Random Forest         | 0.9788   | 0.9952   |
| Dataset 1+2 - XGBoost               | 0.9686   | 0.9993   |
| Dataset 1+2+3 - Logistic Regression | 0.5101   | 0.8854   |
| Dataset 1+2+3 - Random Forest       | 0.9798   | 0.9952   |
| Dataset 1+2+3 - XGBoost             | 0.9738   | 0.9992   |

**Performance Enhancement Analysis:**

**Logistic Regression**: This simple linear model showed no performance enhancement from the amalgamated data. The F1 Score and AUC remained flat across all three datasets. This indicates that the model was unable to find a simple linear relationship between the new features (median_income, Walkability) and the "golden cluster" target.

**Random Forest & XGBoost**: In stark contrast, the more advanced, non-linear models showed a clear and significant performance enhancement with each amalgamation.

On Dataset 1+2, the addition of median_income provided a noticeable lift in the F1 Score, confirming that neighborhood wealth is a valuable predictor.

On Dataset 1+2+3, the further addition of Walkability resulted in the highest F1 Score and AUC. This proves that the combination of socioeconomic and locational convenience data provides the most powerful predictive signal.

Conclusion:

This assignment successfully demonstrates a core principle of machine learning: enriching a dataset with relevant, contextual features is a powerful driver of model performance. While a basic model may not be able to leverage the new information, advanced models like Random Forest and XGBoost can uncover the complex, non-linear patterns within the enriched data. The fully amalgamated Dataset 1+2+3 is definitively the superior dataset, and its value is fully realized when paired with a sophisticated classification algorithm.
